In [1]:
import pdfplumber
import scraper

In [2]:
pdfItikawa = scraper.getPDF("/home/adamson/projects/cs/ref/angus/16_Itikawa_J_Phys_Chem_Ref_Data_35_31_2006.pdf")
pdfBuckman = scraper.getPDF("/home/adamson/projects/cs/ref/angus/16_29_buckman_2003.pdf")

In [3]:
outdir="../../data/raw/ext/n2/itikawa/extracted/"

In [34]:
scraper.removeOutdir(outdir)

In [35]:
cropDim = [(300, 503, 390, 720),(400, 503, 470, 720),(480, 503, 560, 720)]

In [36]:
data = scraper.getTableData(pdf=pdfItikawa,pageNumber=3,cropDimArray=cropDim,locateTables=False,omitRegexp='[0-9]a')
scraper.writeDataToFile([data[0]],outdir+"total_itikawa2006_2a_data")
scraper.writeMetaDataToFile(filename=outdir+"total_itikawa2006_2a",
                   specie="N2",
                   process="total",
                   units_e=1.0,
                   units_sigma=1.0E-20,
                   ref="itikawa2006",
                   background='computed from weighted average of four sets of experimental data, primarily time of flight (TOF) experiments; Itikawa states that the data may have large uncertainty (up to +/- 20%)',
                   lpu=0.2,
                   upu=0.2)

scraper.writeDataToFile([data[1],data[2]],outdir+"total_itikawa2006_2b_data")
scraper.writeMetaDataToFile(filename=outdir+"total_itikawa2006_2b",
                   specie="N2",
                   process="total",
                   units_e=1.0,
                   units_sigma=1.0E-20,
                   ref="itikawa2006",
                   background='computed from weighted average of nine sets of experimental data, primarily time of flight (TOF) experiments; no estimate of the uncertainty'
                   )

In [37]:
#TODO: resonance region total cross section from karwasz2003

In [38]:
cropDim = [(70, 410, 220, 550),(220, 410, 370, 550),(370, 410, 500, 550)]

In [39]:
data = scraper.getTableData(pdf=pdfBuckman,pageNumber=5,cropDimArray=cropDim,locateTables=False)

In [41]:
scraper.writeDataToFile(data,outdir+"elastic_buckman2003_6.2.5.2_data")
scraper.writeMetaDataToFile(filename=outdir+"elastic_buckman2003_6.2.5.2",
                   specie="N2",
                   process="elastic",
                   units_e=1.0,
                   units_sigma=1.0E-20,
                   ref="buckman2003",
                   background='derived from differential scattering measurements; at low energies, in the region of the dominant 2Pig resonance, the level of detail in the DCS measurements does not permit the fine details of the resonance profile to be extracted, so a broad envelope of the resonance enhanced cross section is thus provided; there is good general agreement between the preferred cross section and theoretical calculations',
                   lpu=0.2,
                   upu=0.2)

In [ ]:
#continue from here...

In [ ]:
cropDim[5] = [(300, 105, 390, 333),(390, 105, 480, 333),(480, 105, 570, 333)]

In [ ]:
itikawa_multicolumn_table(5, p6FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [ ]:
cropDim[6] = [(50, 105, 170, 275),(180, 105, 300, 275)]

In [ ]:
itikawa_multicolumn_table(6, p7FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False)

In [ ]:
cropDim[7] = [(52, 541, 300, 721)]

In [ ]:
itikawa_multicolumn_table(7, p8FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False, omitRegexp='[0-9]a')

In [ ]:
def p10FilenameGenerator(i):
    return outdir + "tr_001_" + str(i + 2).zfill(3)

In [ ]:
cropDim[9] = [(120, 440, 200, 750),(210,440,290,750),(290,440,400,750),(400,440,500,750)]

In [ ]:
itikawa_table(9, p10FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)", locateTables=False)

In [ ]:
def p11FilenameGenerator(i):
    return outdir + "tr_001_" + str(i + 6).zfill(3)

In [ ]:
cropDim[10] = [(40, 120, 120, 450),(130,120,210,450),(210,120,290,450),
               (310,120,400,450),(400,120,480,450),(480,120,560,450)]

In [ ]:
itikawa_table(10, p11FilenameGenerator, "Energy(eV),Cross Section(1E-16 cm^2)")